In [1]:
from functools import reduce
import vyper
from openpyxl import load_workbook
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from vyper.user import Model
from vyper.utils.tools import StatisticalTools as st
from vyper.user import Model
from vyper.user.explorer import DataProfiler
from vyper.utils.tools import StatisticalTools as st
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from pandas.api.types import is_string_dtype
from openpyxl import Workbook
import pandas as pd
import numpy as np
import re
import statsmodels.api as sm
import math
import warnings
import string

from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.chart import BarChart, Reference, LineChart
from openpyxl.drawing.text import CharacterProperties
from openpyxl.styles import PatternFill, Border, Side, Alignment, Font

In [2]:
## EDITS
def Snapshot_Profile(profile_data, segment_var, segments, segment_names, include, continuous_var_bounds,
                     variable_order, other_segment = False, file = 'Profile', exclude = '',
                     PPT = True, continuous = [], excludeother = False) -> pd.DataFrame:
    

    profile_data = pd.DataFrame(profile_data)
    
    if (segments is None):
        segments = pd.unique(profile_data[segment_var])
        other_segment = False
        segment_names = str(segments)
    if (not (isinstance(other_segment, bool))):
        raise ValueError('other must be True or False.')
    if (segment_names is None):
        segment_names = str(segments)
        if (other_segment):
            segment_names = [segment_names, 'other']
    if (len(segments)+other_segment!=len(segment_names)):
        if(len(segments)==len(segment_names)):
            segment_names = [segment_names, 'other']
        else:
            warnings.warn('Incorrect length of names, replacing by segments')
            segment_names = str(segments)
        if (other_segment):
            segment_names = [segment_names, 'other']
    ## output -> [1 2 0]

    varclass = pd.DataFrame()
    for col in profile_data.columns:
        varclass = pd.concat([varclass.reset_index(drop=True), pd.DataFrame([st.classify_variable(profile_data[col])], columns = [col])], axis=1)
        
    # Continuous Variables that were classified as categorical
    varclass[varclass.columns in continuous] = 'continuous'
    if (not (include is None)):
        exclude = varclass.columns[varclass.columns not in include]
    # Variables to exclude
    varclass[varclass.columns in exclude] = 'exclude'
    if (excludeother): 
        varclass[varclass.columns =='Categorical+other'] = 'exclude'
    continuous_var_cuts = dict()

    
    for variable in varclass.columns[varclass.iloc[0] == 'continuous']:
        if not continuous_var_bounds:
            continuous_var_bounds = dict()
        if variable in continuous_var_bounds:
            cut_bins = continuous_var_bounds[variable]
            continuous_var_cuts[variable] = pd.cut(profile_data[variable], bins=cut_bins)
                
        else:
            continuous_var_cuts[variable] = st.get_breaks(profile_data.loc[:, variable], nbins=10,squash_extremes=False)

    col_order = []
    index_order = []
    ### TODO: array([2, 0, 1]) Always encode 0 as US
    ### Change the US population as BASELINE
    for seg in segments:
        profile_1 = DataProfiler(profile_data[profile_data[segment_var] == seg]).create_profile(number_of_bins = 10,
                                                                            cts_cuts = continuous_var_cuts)
        if (seg == 'Baseline'):
            profile = profile_1
            profile.columns = ["Variable", "Category",  "US_Count","US_Percent"]
            for cols in profile.columns:
                col_order.append(cols)


        else:
            profile_1.columns = ["Variable", "Category",  "Count_"+str(seg), "Percent_"+str(seg)]
            col_order.append("Count_"+str(seg))
            col_order.append("Percent_"+str(seg))
            profile = pd.merge(profile, profile_1, on = ['Variable', 'Category'], how = 'outer')
            profile['US Population vs ' + "Segment_" + str(seg)] = (profile["US_Percent"]/profile["Percent_"+str(seg)]) * 100
            index_order.append('US Population vs ' + "Segment_" + str(seg))
            
#     for i in range(len(segments)):
#         profile_1 = DataProfiler(profile_data[profile_data[segment_var] == segments[i]]).create_profile(number_of_bins = 10,
#                                                                             cts_cuts = continuous_var_cuts)
#         if (i == 0):
#             profile = profile_1
#             profile.columns = ["Variable", "Category",  "US_Count","US_Percent"]
#             for cols in profile.columns:
#                 col_order.append(cols)


#         else:
#             profile_1.columns = ["Variable", "Category",  "Count_"+str(i), "Percent_"+str(i)]
#             col_order.append("Count_"+str(i))
#             col_order.append("Percent_"+str(i))
#             profile = pd.merge(profile, profile_1, on = ['Variable', 'Category'], how = 'outer')
#             profile['US Population vs ' + "Segment_" + str(i)] = (profile["US_Percent"]/profile["Percent_"+str(i)]) * 100
#             index_order.append('US Population vs ' + "Segment_" + str(i))        

    profile = profile[col_order+index_order]

    for i in profile.columns:

        if ('Count' in i):
            profile[i][profile[i].isna()] = 0
        if ('Percent' in i):
            profile[i][profile[i].isna()] = 0
            
            
    if file:
        import os
        # Make Profile ####
        filesave = file + '.xlsx'
        
        if os.path.exists(filesave):
            os.remove(filesave)
        CreateXLProfile_Snap(profile, savepath = filesave)
        
        if PPT:
            wb = Workbook()

            profile['Category'] = profile['Category'].astype(str)

            all_var_profiling_ws = wb.create_sheet(profile['Variable'].iloc[0], 0)
            # make new sheet the active sheet we are working on
            all_var_profiling_ws = wb.active

            # remove gridlines from the sheet
            all_var_profiling_ws.sheet_view.showGridLines = False

            # set border line thickness
            thick = Side(border_style="thick", color="000000")
            # medium = Side(border_style="medium", color="000000")
            thin = Side(border_style="thin", color="000000")

            # get max rows and cols of profiling df
            max_rows = profile.shape[0]
            max_cols = profile.shape[1]
            
            input_cols = list(string.ascii_uppercase)

            j = 4
            for z in range(0, len(profile)):
                if (z != 0):
                    if (profile['Variable'].iloc[z] != profile['Variable'].iloc[z - 1]):
                        # create sheet
                        all_var_profiling_ws = wb.create_sheet(profile['Variable'].iloc[z], 0)
                        # make new sheet the active sheet we are working on
                        all_var_profiling_ws = wb.active

                        for y in range(len(profile.columns)):
                            current_cell = all_var_profiling_ws.cell(row = 3, column = y + 2)
                            current_cell.font = Font(bold=True)
                            current_cell.alignment = Alignment(horizontal='center', vertical='center')
                            current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
                            current_cell.value = profile.columns[y]
                            if y == 0:
                                current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
                            elif y == max_cols - 1:
                                current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
                            else:
                                current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
                        j = 4
                    else:
                        incol = 2
                        for y in range(len(profile.columns)):
                            current_cell = all_var_profiling_ws.cell(row = 3, column = incol)
                            incol = incol + 1
                            current_cell.font = Font(bold=True)
                            current_cell.alignment = Alignment(horizontal='center', vertical='center')
                            current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
                            current_cell.value = profile.columns[y]
                            if y == 0:
                                current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
                            elif y == max_cols - 1:
                                current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
                            else:
                                current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
                            if (y % 2 == 1 and y > 2):
                                current_cell = all_var_profiling_ws.cell(row = 3, column = incol)
                                incol = incol + 1
                                current_cell.font = Font(bold=True)
                                current_cell.alignment = Alignment(horizontal='center', vertical='center')
                                current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
                                current_cell.value = 'Index'
                                if y == max_cols - 1:
                                    current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
                                else:
                                    current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
                                
                incol = 2
                for y in range(len(profile.columns)):
                    current_cell = all_var_profiling_ws.cell(row = j, column = incol)
                    incol = incol + 1
                    current_cell.value = profile.iloc[z, y]
                    if y == 0:
                        current_cell.border = Border(top=thin, left=thick, right=thin, bottom=thin)
                    else:
                        current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
                    if (y % 2 == 1 and y > 2):
                        current_cell = all_var_profiling_ws.cell(row = j, column = incol)
                        incol = incol + 1
                        current_cell.value = '=' + input_cols[incol - 3] + str(j) + '/E' + str(j)
                        if y == max_cols - 1:
                            current_cell.border = Border(top=thin, left=thin, right=thick, bottom=thin)
                        else:
                            current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

                j = j + 1
            
            filesave = file + 'PPT.xlsx'
        
            if os.path.exists(filesave):
                os.remove(filesave)
            
            wb.save(filesave)
            
    return profile

def CreateXLProfile_Snap(profile, savepath):
    wb = Workbook()

    profile['Category'] = profile['Category'].astype(str)

    # create sheet
    all_var_profiling_ws = wb.create_sheet('profile', 0)

    # make new sheet the active sheet we are working on
    all_var_profiling_ws = wb.active

    # remove gridlines from the sheet
    all_var_profiling_ws.sheet_view.showGridLines = False

    # set border line thickness
    thick = Side(border_style="thick", color="000000")
    # medium = Side(border_style="medium", color="000000")
    thin = Side(border_style="thin", color="000000")

    # get max rows and cols of profiling df
    max_rows = profile.shape[0]
    max_cols = profile.shape[1]

    input_rows = range(4, max_rows + 4)
    input_cols = list(string.ascii_uppercase)[1:max_cols + 1]

    y = 0
    for i in input_cols:
        x = "3"
        x = i + x

        all_var_profiling_ws[x].font = Font(bold=True)
        current_cell = all_var_profiling_ws[x]
        current_cell.alignment = Alignment(horizontal='center', vertical='center')
        current_cell.fill = PatternFill("solid", fgColor="A9C4FE")

        if y == 0:
            current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
        elif y == max_cols - 1:
            current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
        else:
            current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)

        all_var_profiling_ws[x] = profile.columns[y]
        y = y + 1
    
    print("Worked till here")
    counter = []
    counter_val = 1
    for index,i in enumerate(input_cols[2::]):
        x = "2"
        x = i + x
        if index%2==1:
            all_var_profiling_ws.merge_cells(f"{counter[-1]}:{x}")
            all_var_profiling_ws[counter[-1]].font = Font(bold=True)
            current_cell = all_var_profiling_ws[counter[-1]]
            current_cell.alignment = Alignment(horizontal='center', vertical='center')
            current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
            continue


        if i==input_cols[-1]:
            break
        if i=='D':
            all_var_profiling_ws[x] = "US POPULATION"
            all_var_profiling_ws.merge_cells(start_row=2, start_column=4, end_row=2, end_column=5)
            counter.append(x)
            continue
        all_var_profiling_ws[x] = 'Segment ' + str(counter_val)
        counter_val+=1
        counter.append(x)
        # all_var_profiling_ws.merge_cells(start_row=2, start_column=5, end_row=2, end_column=5)
        # all_var_profiling_ws.merge_cells(f'{x}:D2')

    
    def style_range(ws, cell_range, border=Border(), fill=None, font=None,alignment=None):
        
        """
        Apply styles to a range of cells as if they were a single cell.

        :param ws:  Excel worksheet instance
        :param range: An excel range to style (e.g. A1:F20)
        :param border: An openpyxl Border
        :param fill: An openpyxl PatternFill or GradientFill
        :param font: An openpyxl Font object
        """

        top = Border(top=border.top)
        left = Border(left=border.left)
        right = Border(right=border.right)
        bottom = Border(bottom=border.bottom)

        first_cell = ws[cell_range.split(":")[0]]
        if alignment:
            ws.merge_cells(cell_range)
            first_cell.alignment = alignment

        rows = ws[cell_range]
        if font:
            first_cell.font = font

        for cell in rows[0]:
            cell.border = cell.border + top
        for cell in rows[-1]:
            cell.border = cell.border + bottom

        for row in rows:
            l = row[0]
            r = row[-1]
            l.border = l.border + left
            r.border = r.border + right
            if fill:
                for c in row:
                    c.fill = fill


    border = Border(top=thick, left=thick, right=thick, bottom=thick)

    for range1 in all_var_profiling_ws.merged_cells.ranges:
        style_range(all_var_profiling_ws, str(range1), border=border)

    

    y = 0
    for i in input_cols:
        z = 0
        for j in input_rows:
            all_var_profiling_ws.row_dimensions[j].height = 25
            x = str(j)
            x = i + x
            # print(x)
            current_cell = all_var_profiling_ws[x]
            current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

            if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
                current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)

            all_var_profiling_ws[x] = profile.iloc[z, y]

            if y == 0:
                current_cell.border = Border(top=thin, left=thick, right=thin, bottom=thin)
                if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
                    current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
            elif y == max_cols - 1:
                current_cell.border = Border(top=thin, left=thin, right=thick, bottom=thin)
                if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
                    current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)


            z = z + 1
        y = y + 1
    

    
    if (savepath != ''):
        wb.save(savepath)

In [19]:
# Read in the demonstration Epsilon Response Model Data - Internal Append
eps_data = pd.read_csv(r"Data/EpsilonResponseModelData_InternalAppend.csv")

# # Create random segments to test snapshot
# eps_data['seg'] = np.random.randint(0, 3, eps_data.shape[0])

# Output length to confirm read in
len(eps_data)

99260

In [20]:
# Read in the Epsilon Data Dictionary we created
epsilon_dict = pd.read_csv('Data/EpsilonDataDictionary.csv')

# Pull list of Selected Attributes from Snowflake Headers
variable_list = epsilon_dict['Snowflake Headers'].tolist()
# variable_list.append('seg')

# Selected Attributes only
new_df = eps_data[variable_list]
new_df

,RECORD_QUALITY_CODE,NUM_SOURC_VERIFY_HH,FIPS_STATE_CODE,ZIP,ZIP4,DELIVERY_POINT_CODE,CARRIER_ROUTE,CONTRACTED_ADDRESS,POST_OFFICE_NAME,STATE,...,PROPERTY_LOT_SIZE_IN_ACRES_DUPLICATE,MT_TRADITIONAL_CABLE_PAY_TV_SERVICE_CUSTOMERS,INVESTMENT_RESOURCES_TIERS,LIQUID_RESOURCES_TIERS,MORTGAGE_LIABILITY_TIERS,SHORT_TERM_LIABILITY_TIERS,WEALTH_RESOURCES_TIERS,_2010_CENSUS_GEO_TRAC_BLOCK_GROUP,CENSUS_TRACT_SUFFIX,_2010_CENSUS_GEO_BLOCK
0,1,9,36,11518,1301,135,C001,113 CARMAN AVE,EAST ROCKAWAY,NY,...,NaN,13,11,14,NaN,6,7,4121005,0,3
1,1,18,36,11577,1984,476,C017,147 RED POLE WAY,ROSLYN HTS,NY,...,I,1,13,14,123.0,7,10,3009005,0,2
2,1,10,9,6905,4453,590,C003,259 BRIDGE ST,STAMFORD,CT,...,NaN,96,10,10,NaN,5,5,213002,0,7
3,1,10,36,10128,6847,959,C126,408 E 92ND ST APT 29E,NEW YORK,NY,...,NaN,63,11,12,NaN,6,6,152003,0,0
4,1,23,34,7446,1736,570,C014,357 WOODLAND AVE,RAMSEY,NJ,...,D,4,12,14,NaN,4,7,441002,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99255,1,3,36,11694,1328,87,C011,508 BEACH 136TH ST,ROCKAWAY PARK,NY,...,NaN,67,11,11,NaN,7,6,928002,0,10
99256,1,18,34,7920,2529,59,C004,105 TUXFORD TER,BASKING RIDGE,NJ,...,E,8,13,14,360.0,1,10,526012,1,26
99257,2,18,36,10701,5372,996,C017,21 VICTORIA LN,YONKERS,NY,...,NaN,24,6,6,NaN,5,5,17002,0,15
99258,1,23,34,7446,1422,848,C022,84 WARREN ST,RAMSEY,NJ,...,A,17,13,14,495.0,8,10,441001,0,15


In [21]:
# Split the data in half just for testing purposes of dummy groups
cutoff = int(len(new_df)/2)
baseline_df = new_df.iloc[:cutoff,:]
baseline_df = new_df.iloc[:cutoff,:]
customer1_df = new_df.iloc[cutoff:,:]

# Create Dummy Groups
baseline_df['Group'] = 'Baseline'
customer1_df['Group'] = 'Customer_1'
             
# Merge the two dataframes together again
final_df = baseline_df.append(customer1_df)
final_df

,RECORD_QUALITY_CODE,NUM_SOURC_VERIFY_HH,FIPS_STATE_CODE,ZIP,ZIP4,DELIVERY_POINT_CODE,CARRIER_ROUTE,CONTRACTED_ADDRESS,POST_OFFICE_NAME,STATE,...,MT_TRADITIONAL_CABLE_PAY_TV_SERVICE_CUSTOMERS,INVESTMENT_RESOURCES_TIERS,LIQUID_RESOURCES_TIERS,MORTGAGE_LIABILITY_TIERS,SHORT_TERM_LIABILITY_TIERS,WEALTH_RESOURCES_TIERS,_2010_CENSUS_GEO_TRAC_BLOCK_GROUP,CENSUS_TRACT_SUFFIX,_2010_CENSUS_GEO_BLOCK,Group
0,1,9,36,11518,1301,135,C001,113 CARMAN AVE,EAST ROCKAWAY,NY,...,13,11,14,NaN,6,7,4121005,0,3,Baseline
1,1,18,36,11577,1984,476,C017,147 RED POLE WAY,ROSLYN HTS,NY,...,1,13,14,123.0,7,10,3009005,0,2,Baseline
2,1,10,9,6905,4453,590,C003,259 BRIDGE ST,STAMFORD,CT,...,96,10,10,NaN,5,5,213002,0,7,Baseline
3,1,10,36,10128,6847,959,C126,408 E 92ND ST APT 29E,NEW YORK,NY,...,63,11,12,NaN,6,6,152003,0,0,Baseline
4,1,23,34,7446,1736,570,C014,357 WOODLAND AVE,RAMSEY,NJ,...,4,12,14,NaN,4,7,441002,0,5,Baseline
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99255,1,3,36,11694,1328,87,C011,508 BEACH 136TH ST,ROCKAWAY PARK,NY,...,67,11,11,NaN,7,6,928002,0,10,Customer_1
99256,1,18,34,7920,2529,59,C004,105 TUXFORD TER,BASKING RIDGE,NJ,...,8,13,14,360.0,1,10,526012,1,26,Customer_1
99257,2,18,36,10701,5372,996,C017,21 VICTORIA LN,YONKERS,NY,...,24,6,6,NaN,5,5,17002,0,15,Customer_1
99258,1,23,34,7446,1422,848,C022,84 WARREN ST,RAMSEY,NJ,...,17,13,14,495.0,8,10,441001,0,15,Customer_1


In [22]:
len(final_df)

99260

In [6]:
# # Split the data in thres just for testing purposes of dummy groups
# cutoff = int(len(new_df)/3)
# baseline_df = new_df.iloc[:cutoff,:]
# customer1_df = new_df.iloc[cutoff:cutoff*2,:]
# customer2_df = new_df.iloc[cutoff*2:,:]             

# # # # Create Dummy Groups
# # # baseline_df['Group'] = 'Baseline'
# # # customer1_df['Group'] = 'Customer_1'
# # # customer2_df['Group'] = 'Customer_2'
             
# # # # Merge the two dataframes together again
# # # final_df = baseline_df.append(customer1_df)
# # # final_df = final_df.append(customer2_df)
# # # final_df

In [7]:
# Function to combine customer data with baseline and add the Group segment
def append_data(baseline_df, *customer_dfs):
    # Tag baseline group segment
    baseline_df['Group'] = "Baseline"
    append_df = baseline_df
    
    # Start a counter for group segmentation
    customer_count = 1
    
    # Loop over the customer data, adding the group segment then appending
    for customer_df in customer_dfs:
        customer_df['Group'] = f'Customer {str(customer_count)}'
        customer_count += 1
        append_df = append_df.append(customer_df)
        
    append_df.reset_index(inplace=True)
        
    return append_df

In [8]:
test_df = append_data(new_df, customer1_df)
test_df

,index,RECORD_QUALITY_CODE,NUM_SOURC_VERIFY_HH,FIPS_STATE_CODE,ZIP,ZIP4,DELIVERY_POINT_CODE,CARRIER_ROUTE,CONTRACTED_ADDRESS,POST_OFFICE_NAME,...,MT_TRADITIONAL_CABLE_PAY_TV_SERVICE_CUSTOMERS,INVESTMENT_RESOURCES_TIERS,LIQUID_RESOURCES_TIERS,MORTGAGE_LIABILITY_TIERS,SHORT_TERM_LIABILITY_TIERS,WEALTH_RESOURCES_TIERS,_2010_CENSUS_GEO_TRAC_BLOCK_GROUP,CENSUS_TRACT_SUFFIX,_2010_CENSUS_GEO_BLOCK,Group
0,0,1,9,36,11518,1301,135,C001,113 CARMAN AVE,EAST ROCKAWAY,...,13,11,14,NaN,6,7,4121005,0,3,Baseline
1,1,1,18,36,11577,1984,476,C017,147 RED POLE WAY,ROSLYN HTS,...,1,13,14,123.0,7,10,3009005,0,2,Baseline
2,2,1,10,9,6905,4453,590,C003,259 BRIDGE ST,STAMFORD,...,96,10,10,NaN,5,5,213002,0,7,Baseline
3,3,1,10,36,10128,6847,959,C126,408 E 92ND ST APT 29E,NEW YORK,...,63,11,12,NaN,6,6,152003,0,0,Baseline
4,4,1,23,34,7446,1736,570,C014,357 WOODLAND AVE,RAMSEY,...,4,12,14,NaN,4,7,441002,0,5,Baseline
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148885,99255,1,3,36,11694,1328,87,C011,508 BEACH 136TH ST,ROCKAWAY PARK,...,67,11,11,NaN,7,6,928002,0,10,Customer 1
148886,99256,1,18,34,7920,2529,59,C004,105 TUXFORD TER,BASKING RIDGE,...,8,13,14,360.0,1,10,526012,1,26,Customer 1
148887,99257,2,18,36,10701,5372,996,C017,21 VICTORIA LN,YONKERS,...,24,6,6,NaN,5,5,17002,0,15,Customer 1
148888,99258,1,23,34,7446,1422,848,C022,84 WARREN ST,RAMSEY,...,17,13,14,495.0,8,10,441001,0,15,Customer 1


In [9]:
len(test_df)

148890

In [10]:
# Function to read in a csv with containing the variable names and the respective bin cutoffs
def continuous_bins(filepath):
    df_continuous = pd.read_csv(filepath)
    
    continuous_bounds = {}
    for index, row in df_continuous.iterrows():
        continuous_bounds[row['Snowflake Field names']] = [float(i) for i in row['Bin'].split(sep=',')]
        
    return continuous_bounds

In [11]:
# Set up variables for snapshot
file_name = '2-segment-append-test'
seg_var = 'Group'
dataset = test_df

# Read in file and set bins
bin_vars = continuous_bins('Data/bin_dictionary.csv')

In [12]:
# Run Snapshot Report
profile = Snapshot_Profile(profile_data = dataset, segment_var=seg_var, segments = None, segment_names = None, include = None,
                 continuous_var_bounds = bin_vars, variable_order = None, other_segment = False,
                 file = file_name, exclude = [], PPT = True, continuous = [], excludeother = False)
profile

Worked till here


,Variable,Category,US_Count,US_Percent,Count_Customer 1,Percent_Customer 1,US Population vs Segment_Customer 1
0,index,"(-0.001, 14888.9]",14889,15.000000,0.0,0.000000,NaN
1,index,"(14888.9, 29777.8]",14889,15.000000,0.0,0.000000,NaN
2,index,"(29777.8, 44666.7]",14889,15.000000,0.0,0.000000,NaN
3,index,"(44666.7, 54592.6]",9926,10.000000,4963.0,10.000000,100.000000
4,index,"(54592.6, 62037.0]",7445,7.500504,7445.0,15.001007,50.000000
...,...,...,...,...,...,...,...
6749,_2010_CENSUS_GEO_BLOCK,"(5.0, 7.0]",7492,7.547854,3824.0,7.705017,97.960251
6750,_2010_CENSUS_GEO_BLOCK,"(7.0, 10.0]",9496,9.566794,4738.0,9.546645,100.211060
6751,_2010_CENSUS_GEO_BLOCK,"(10.0, 14.0]",9243,9.311908,4610.0,9.288737,100.249458
6752,_2010_CENSUS_GEO_BLOCK,"(14.0, 21.0]",9880,9.953657,4953.0,9.979851,99.737533


In [13]:
file_xlsx = file_name + '.xlsx'
CreateXLProfile_Snap(profile, file_xlsx)

Worked till here


In [14]:
# def Snapshot_Profile(profile_data, segment_var, segments, segment_names, include, continuous_var_bounds,
#                      variable_order, other_segment = False, file = 'Profile', exclude = '',
#                      PPT = True, continuous = [], excludeother = False) -> pd.DataFrame:
#     profile_data = pd.DataFrame(profile_data)
#     if (segments is None):
#         segments = pd.unique(profile_data[segment_var])
#         other_segment = False
#         segment_names = str(segments)
#     if (not (isinstance(other_segment, bool))):
#         raise ValueError('other must be True or False.')
#     if (segment_names is None):
#         segment_names = str(segments)
#         if (other_segment):
#             segment_names = [segment_names, 'other']
#     if (len(segments)+other_segment!=len(segment_names)):
#         if(len(segments)==len(segment_names)):
#             segment_names = [segment_names, 'other']
#         else:
#             warnings.warn('Incorrect length of names, replacing by segments')
#             segment_names = str(segments)
#         if (other_segment):
#             segment_names = [segment_names, 'other']

#     varclass = pd.DataFrame()
#     for col in profile_data.columns:
#         varclass = pd.concat([varclass.reset_index(drop=True), pd.DataFrame([st.classify_variable(profile_data[col])], columns = [col])], axis=1)

#     # Continuous Variables that were classified as categorical
#     varclass[varclass.columns in continuous] = 'continuous'
#     if (not (include is None)):
#         exclude = varclass.columns[varclass.columns not in include]
#     # Variables to exclude
#     varclass[varclass.columns in exclude] = 'exclude'
#     if (excludeother): varclass[varclass.columns =='Categorical+other'] = 'exclude'


#     continuous_var_cuts = dict()
#     for variable in varclass.columns[varclass.iloc[0] == 'continuous']:
#         if not continuous_var_bounds:
#             continuous_var_bounds = dict()
#         if variable in continuous_var_bounds:
#             continuous_var_cuts[variable] = continuous_var_bounds[variable]
#         else:
#             continuous_var_cuts[variable] = st.get_breaks(profile_data.loc[:, variable], nbins=10,
#                      squash_extremes=False)

#     for i in range(len(segments)):
#         profile_1 = DataProfiler(profile_data[profile_data[segment_var] == segments[i]]).create_profile(number_of_bins = 10,
#                                                                           cts_cuts = continuous_var_cuts)
#         if (i == 0):
#             profile = profile_1
#         else:
#             profile = pd.merge(profile, profile_1, on = ['Variable', 'Category'], how = 'outer')

#     for i in profile.columns:
#         print(i)
#         print(profile[i])
#         if ('Count' in i):
#             profile[i][profile[i].isna()] = 0
#         if ('Percent' in i):
#             profile[i][profile[i].isna()] = 0
            
            
#     if file:
#         import os
#         # Make Profile ####
#         filesave = file + '.xlsx'
        
#         if os.path.exists(filesave):
#             os.remove(filesave)
#         CreateXLProfile_Snap(profile, savepath = filesave)
        
#         if PPT:
#             wb = Workbook()

#             profile['Category'] = profile['Category'].astype(str)

#             all_var_profiling_ws = wb.create_sheet(profile['Variable'].iloc[0], 0)
#             # make new sheet the active sheet we are working on
#             all_var_profiling_ws = wb.active

#             # remove gridlines from the sheet
#             all_var_profiling_ws.sheet_view.showGridLines = False

#             # set border line thickness
#             thick = Side(border_style="thick", color="000000")
#             # medium = Side(border_style="medium", color="000000")
#             thin = Side(border_style="thin", color="000000")

#             # get max rows and cols of profiling df
#             max_rows = profile.shape[0]
#             max_cols = profile.shape[1]
            
#             input_cols = list(string.ascii_uppercase)

#             j = 4
#             for z in range(0, len(profile)):
#                 if (z != 0):
#                     if (profile['Variable'].iloc[z] != profile['Variable'].iloc[z - 1]):
#                         # create sheet
#                         all_var_profiling_ws = wb.create_sheet(profile['Variable'].iloc[z], 0)
#                         # make new sheet the active sheet we are working on
#                         all_var_profiling_ws = wb.active

#                         for y in range(len(profile.columns)):
#                             current_cell = all_var_profiling_ws.cell(row = 3, column = y + 2)
#                             current_cell.font = Font(bold=True)
#                             current_cell.alignment = Alignment(horizontal='center', vertical='center')
#                             current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
#                             current_cell.value = profile.columns[y]
#                             if y == 0:
#                                 current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
#                             elif y == max_cols - 1:
#                                 current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
#                             else:
#                                 current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
#                         j = 4
#                     else:
#                         incol = 2
#                         for y in range(len(profile.columns)):
#                             current_cell = all_var_profiling_ws.cell(row = 3, column = incol)
#                             incol = incol + 1
#                             current_cell.font = Font(bold=True)
#                             current_cell.alignment = Alignment(horizontal='center', vertical='center')
#                             current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
#                             current_cell.value = profile.columns[y]
#                             if y == 0:
#                                 current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
#                             elif y == max_cols - 1:
#                                 current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
#                             else:
#                                 current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
#                             if (y % 2 == 1 and y > 2):
#                                 current_cell = all_var_profiling_ws.cell(row = 3, column = incol)
#                                 incol = incol + 1
#                                 current_cell.font = Font(bold=True)
#                                 current_cell.alignment = Alignment(horizontal='center', vertical='center')
#                                 current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
#                                 current_cell.value = 'Index'
#                                 if y == max_cols - 1:
#                                     current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
#                                 else:
#                                     current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
                                
#                 incol = 2
#                 for y in range(len(profile.columns)):
#                     current_cell = all_var_profiling_ws.cell(row = j, column = incol)
#                     incol = incol + 1
#                     current_cell.value = profile.iloc[z, y]
#                     if y == 0:
#                         current_cell.border = Border(top=thin, left=thick, right=thin, bottom=thin)
#                     else:
#                         current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
#                     if (y % 2 == 1 and y > 2):
#                         current_cell = all_var_profiling_ws.cell(row = j, column = incol)
#                         incol = incol + 1
#                         current_cell.value = '=' + input_cols[incol - 3] + str(j) + '/E' + str(j)
#                         if y == max_cols - 1:
#                             current_cell.border = Border(top=thin, left=thin, right=thick, bottom=thin)
#                         else:
#                             current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

#                 j = j + 1
            
#             filesave = file + 'PPT.xlsx'
        
#             if os.path.exists(filesave):
#                 os.remove(filesave)
            
#             wb.save(filesave)
            
#     return profile

In [15]:
# def CreateXLProfile_Snap(profile, savepath):
#     wb = Workbook()

#     profile['Category'] = profile['Category'].astype(str)

#     # create sheet
#     all_var_profiling_ws = wb.create_sheet('profile', 0)

#     # make new sheet the active sheet we are working on
#     all_var_profiling_ws = wb.active

#     # remove gridlines from the sheet
#     all_var_profiling_ws.sheet_view.showGridLines = False

#     # set border line thickness
#     thick = Side(border_style="thick", color="000000")
#     # medium = Side(border_style="medium", color="000000")
#     thin = Side(border_style="thin", color="000000")

#     # get max rows and cols of profiling df
#     max_rows = profile.shape[0]
#     max_cols = profile.shape[1]

#     input_rows = range(4, max_rows + 4)
#     input_cols = list(string.ascii_uppercase)[1:max_cols + 1]

#     y = 0
#     for i in input_cols:
#         x = "3"
#         x = i + x

#         all_var_profiling_ws[x].font = Font(bold=True)
#         current_cell = all_var_profiling_ws[x]
#         current_cell.alignment = Alignment(horizontal='center', vertical='center')
#         current_cell.fill = PatternFill("solid", fgColor="A9C4FE")

#         if y == 0:
#             current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
#         elif y == max_cols - 1:
#             current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
#         else:
#             current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)

#         all_var_profiling_ws[x] = profile.columns[y]
#         y = y + 1

#     y = 0
#     for i in input_cols:
#         z = 0
#         for j in input_rows:
#             all_var_profiling_ws.row_dimensions[j].height = 25
#             x = str(j)
#             x = i + x
#             # print(x)
#             current_cell = all_var_profiling_ws[x]
#             current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

#             if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
#                 current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)

#             all_var_profiling_ws[x] = profile.iloc[z, y]

#             if y == 0:
#                 current_cell.border = Border(top=thin, left=thick, right=thin, bottom=thin)
#                 if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
#                     current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
#             elif y == max_cols - 1:
#                 current_cell.border = Border(top=thin, left=thin, right=thick, bottom=thin)
#                 if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
#                     current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)


#             z = z + 1
#         y = y + 1
    

    
#     if (savepath != ''):
#         wb.save(savepath)